In [163]:
import pandas as pd
import re
import nltk
import string

#nltk.download('punkt')
#nltk.download('stopwords')

In [2]:
NLP_clin_note=pd.read_csv("ClinNotes.csv")

In [3]:
NLP_clin_note.head()

,category,notes
0,Cardiovascular / Pulmonary,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
1,Cardiovascular / Pulmonary,1. The left ventricular cavity size and wall ...
2,Cardiovascular / Pulmonary,"2-D ECHOCARDIOGRAM,Multiple views of the heart..."
3,Cardiovascular / Pulmonary,"DESCRIPTION:,1. Normal cardiac chambers size...."
4,Cardiovascular / Pulmonary,"2-D STUDY,1. Mild aortic stenosis, widely calc..."


In [4]:
NLP_clin_note.describe()

,category,notes
count,818,818
unique,3,812
top,Cardiovascular / Pulmonary,"PREOPERATIVE DIAGNOSES:,1. Carotid artery occ..."
freq,371,2


In [7]:
NLP_clin_note.describe().sum(axis=1)

count                                                  1636
unique                                                  815
top       Cardiovascular / PulmonaryPREOPERATIVE DIAGNOS...
freq                                                    373
dtype: object

In [8]:
data_notes=NLP_clin_note.notes.values

In [9]:
len(data_notes)

818

In [10]:
#Clean up

In [18]:
#Using regular expression deleting the ip address

In [19]:
data_notes_noip = [re.sub('[\d+\.{3}]\d+',"",txt) for txt in data_notes]

In [20]:
#normalizing case

In [21]:
data_notes_lower = [txt.lower() for txt in data_notes_noip]

In [22]:
#Using regular expression removing URLs

In [23]:
data_notes_nourl = [re.sub("\w+://\S+","", txt) for txt in data_notes_lower]

In [24]:
#removing extra line break

In [25]:
data_notes_nourl = [txt.replace("\'","") for txt in data_notes_nourl]

In [26]:
#Tokenization

In [27]:
from nltk.tokenize import word_tokenize

In [28]:
data_notes_tokens = [word_tokenize(sent) for sent in data_notes_nourl ]

print(data_notes_tokens[0])

['2-d', 'm-mode', ':', ',', ',1.', 'left', 'atrial', 'enlargement', 'with', 'left', 'atrial', 'diameter', 'of', '4', 'cm.,2', '.', 'normal', 'size', 'right', 'and', 'left', 'ventricle.,3', '.', 'normal', 'lv', 'systolic', 'function', 'with', 'left', 'ventricular', 'ejection', 'fraction', 'of', '%', '.,4', '.', 'normal', 'lv', 'diastolic', 'function.,5', '.', 'no', 'pericardial', 'effusion.,6', '.', 'normal', 'morphology', 'of', 'aortic', 'valve', ',', 'mitral', 'valve', ',', 'tricuspid', 'valve', ',', 'and', 'pulmonary', 'valve.,7', '.', 'pa', 'systolic', 'pressure', 'is', 'mmhg.', ',', 'doppler', ':', ',', ',1.', 'mild', 'mitral', 'and', 'tricuspid', 'regurgitation.,2', '.', 'trace', 'aortic', 'and', 'pulmonary', 'regurgitation', '.']


In [29]:
#Deleting stopwords and punctuations

In [30]:
from nltk.corpus import stopwords
from string import punctuation

In [31]:
stop_nltk = stopwords.words("english")
stop_punct = list(punctuation)

In [47]:
stop_final = stop_nltk + stop_punct + ["...", "``","''", "====", "must"]

In [48]:
def clean (sent):
    return [word for word in sent if word not in stop_final]

In [49]:

clean(data_notes_tokens[0])


['2-d',
 'm-mode',
 ',1.',
 'left',
 'atrial',
 'enlargement',
 'left',
 'atrial',
 'diameter',
 '4',
 'cm.,2',
 'normal',
 'size',
 'right',
 'left',
 'ventricle.,3',
 'normal',
 'lv',
 'systolic',
 'function',
 'left',
 'ventricular',
 'ejection',
 'fraction',
 '.,4',
 'normal',
 'lv',
 'diastolic',
 'function.,5',
 'pericardial',
 'effusion.,6',
 'normal',
 'morphology',
 'aortic',
 'valve',
 'mitral',
 'valve',
 'tricuspid',
 'valve',
 'pulmonary',
 'valve.,7',
 'pa',
 'systolic',
 'pressure',
 'mmhg.',
 'doppler',
 ',1.',
 'mild',
 'mitral',
 'tricuspid',
 'regurgitation.,2',
 'trace',
 'aortic',
 'pulmonary',
 'regurgitation']

In [50]:
data_notes_clean = [clean(sent) for sent in data_notes_tokens]

In [51]:
#Checking out the top terms in the data

In [52]:
from collections import Counter

In [53]:
word_list = []
for sent in data_notes_clean:
    word_list.extend(sent)

In [54]:
res = Counter(word_list)
res.most_common(20)

[('patient', 3041),
 ('left', 2301),
 ('right', 2105),
 ('history', 1403),
 ('normal', 1342),
 ('procedure', 1180),
 ('artery', 1176),
 ('mg', 834),
 ('placed', 828),
 ('well', 752),
 ('coronary', 698),
 ('also', 647),
 ('chest', 632),
 ('performed', 622),
 ('pain', 616),
 ('without', 585),
 (',1.', 568),
 ('time', 568),
 ('blood', 555),
 ('1', 532)]

In [60]:
#Contextual stop words - "patient", "left", "right", "history", "normal", "procedure", "artery"

In [61]:
Highly_Repeated = ["patient", "left", "right", "history", "normal", "procedure", "artery"]

In [57]:
Highly_Repeated = stop_final + Highly_Repeated 

In [58]:
word_list = [clean(sent) for sent in data_notes_tokens]

In [59]:
word_list = [" ".join(sent) for sent in data_notes_clean]
word_list[:2]

['2-d m-mode ,1. left atrial enlargement left atrial diameter 4 cm.,2 normal size right left ventricle.,3 normal lv systolic function left ventricular ejection fraction .,4 normal lv diastolic function.,5 pericardial effusion.,6 normal morphology aortic valve mitral valve tricuspid valve pulmonary valve.,7 pa systolic pressure mmhg. doppler ,1. mild mitral tricuspid regurgitation.,2 trace aortic pulmonary regurgitation',
 '1. left ventricular cavity size wall thickness appear normal wall motion left ventricular systolic function appears hyperdynamic estimated ejection fraction near-cavity obliteration seen also appears increased left ventricular outflow tract gradient mid cavity level consistent hyperdynamic left ventricular systolic function abnormal left ventricular relaxation pattern seen well elevated left atrial pressures seen doppler examination.,2 left atrium appears mildly dilated.,3 right atrium right ventricle appear normal.,4 aortic root appears normal.,5 aortic valve appear

In [71]:
# This function will be applied later on the test set

In [72]:
#Applying Word2Vec library to find out clinical terms

In [64]:
from gensim.models import Word2Vec

In [65]:
import warnings
warnings.filterwarnings('ignore')

In [66]:
mymodel = Word2Vec(data_notes_clean, min_count=1)

In [67]:
#Summerizing the loaded model
print(mymodel)

Word2Vec(vocab=17478, size=100, alpha=0.025)


In [70]:
#Here the number of unique vocabulary is 17478 and with 100 value size.
#Hence, all the 17478 vocabs will be rated in 100 features.

In [73]:
#Summerizing the vocabulary
words = list(mymodel.wv.vocab)

In [74]:
print(words)

['2-d', 'm-mode', ',1.', 'left', 'atrial', 'enlargement', 'diameter', '4', 'cm.,2', 'normal', 'size', 'right', 'ventricle.,3', 'lv', 'systolic', 'function', 'ventricular', 'ejection', 'fraction', '.,4', 'diastolic', 'function.,5', 'pericardial', 'effusion.,6', 'morphology', 'aortic', 'valve', 'mitral', 'tricuspid', 'pulmonary', 'valve.,7', 'pa', 'pressure', 'mmhg.', 'doppler', 'mild', 'regurgitation.,2', 'trace', 'regurgitation', '1.', 'cavity', 'wall', 'thickness', 'appear', 'motion', 'appears', 'hyperdynamic', 'estimated', 'near-cavity', 'obliteration', 'seen', 'also', 'increased', 'outflow', 'tract', 'gradient', 'mid', 'level', 'consistent', 'abnormal', 'relaxation', 'pattern', 'well', 'elevated', 'pressures', 'examination.,2', 'atrium', 'mildly', 'dilated.,3', 'ventricle', 'normal.,4', 'root', 'normal.,5', 'calcified', 'stenosis', 'calculated', 'area', '1', 'cm', 'square', 'maximum', 'instantaneous', 'mean', 'mm.,6', 'annular', 'calcification', 'extending', 'leaflets', 'supportive'

In [75]:
#Access word vectorfor one word "allergy"
print(mymodel['allergy'])

[ 0.03106281 -0.07074327  0.04528017  0.16558787  0.08254202  0.10887036
  0.03197986  0.1958983  -0.10656888  0.0334806   0.03862099 -0.04223233
 -0.04572555 -0.10445784  0.05143708 -0.04712089  0.08446626 -0.1325916
  0.0968876  -0.02690332  0.06379066  0.01096087  0.01100656 -0.01099656
 -0.01176183  0.13956495 -0.03462233  0.07849466  0.21987307  0.10976051
  0.04065366 -0.02805992 -0.02344968 -0.12820625  0.03302466  0.04898654
  0.06109745  0.02393153  0.05513814  0.00918279 -0.03220907  0.05300608
  0.0023952   0.15712751  0.0936674  -0.05891731  0.11208563  0.01157327
  0.12993358 -0.18160485 -0.09727473  0.01755428 -0.04410746  0.03760902
 -0.04152933  0.13273336  0.02738803 -0.07390727  0.10324132  0.1178416
 -0.05794292  0.23620804 -0.02957127  0.08089279 -0.13097455  0.19013423
  0.09664432  0.03826337 -0.05076467  0.04340009  0.0281681  -0.11033987
 -0.00567356  0.07536107 -0.0665089  -0.03894016 -0.15872507 -0.02581827
  0.022851    0.14955792 -0.02432656 -0.02895778 -0.0

In [77]:
#The word "allergy" has been rated 100 different features here, which are the values.

In [87]:
data2=pd.read_csv("MedicalConcepts.csv",)
data2.head(20)

,Term1,Term2
0,Glaucoma,Fibrillation
1,Carbatrol,Dilantin
2,Cardiomyopathy,Tylenol
3,Herpes,Hyperthyroidism
4,Seasickness,Nausea
5,Coccidioidomycosis,Histoplasmosis
6,Angina,Plavix
7,Zofran,Ipecac Syrup
8,Seizures,Insulin
9,Septicemia,Dyspnea


In [88]:
data2['Term1'].value_counts()

Fibrillation          8
Mycosis               8
Dyslipidemia          8
Cataracts             7
Cirrhosis             7
                     ..
Dementia              1
Ceftiaxone            1
Dysentery             1
Glyburide             1
Glomerulosclerosis    1
Name: Term1, Length: 269, dtype: int64

In [89]:
data2_words1= data2['Term1']

In [90]:
data2_token1 = [word_tokenize(word) for word in data2_words1]

print(data2_token1)

[['Glaucoma'], ['Carbatrol'], ['Cardiomyopathy'], ['Herpes'], ['Seasickness'], ['Coccidioidomycosis'], ['Angina'], ['Zofran'], ['Seizures'], ['Septicemia'], ['Nystagmus'], ['Seasickness'], ['Earache'], ['Plague'], ['Zovirax'], ['Nausea'], ['Ketonuria'], ['Convulsion'], ['Parasitemia'], ['Proteinuria'], ['Gastroenteritis'], ['Hypoproteinemia'], ['Spiriva'], ['Ketonuria'], ['Insulin'], ['Cataracts'], ['Methadone'], ['Diabetes'], ['Blanching'], ['Thirsty'], ['Photosensitization'], ['Polyuria'], ['Psoriasis'], ['Virilism'], ['Comatose'], ['Sleeplessness'], ['Neuropathy'], ['Psoriasis'], ['Tuberculosis'], ['Obesity'], ['Thromboembolism'], ['Pneumoniae'], ['Cardura'], ['Cataracts'], ['Cirrhosis'], ['Seizures'], ['Cataracts'], ['Anemia'], ['Convulsion'], ['Osteoporosis'], ['Seizures'], ['Garlic'], ['Dyslipidemia'], ['Fibrillation'], ['Cefoxitin'], ['Deafness'], ['Penicillin'], ['Diabetes'], ['Hemochromatosis'], ['Ischemias'], ['Pallor'], ['Earache'], ['Herpes'], ['Dyslipidemia'], ['Rabies'], 

In [95]:
mymodel_2= Word2Vec(data2_token1, min_count=1)

In [96]:
print(mymodel_2)

Word2Vec(vocab=273, size=100, alpha=0.025)


In [97]:
words_2 = list(mymodel_2.wv.vocab)
print(words_2)

['Glaucoma', 'Carbatrol', 'Cardiomyopathy', 'Herpes', 'Seasickness', 'Coccidioidomycosis', 'Angina', 'Zofran', 'Seizures', 'Septicemia', 'Nystagmus', 'Earache', 'Plague', 'Zovirax', 'Nausea', 'Ketonuria', 'Convulsion', 'Parasitemia', 'Proteinuria', 'Gastroenteritis', 'Hypoproteinemia', 'Spiriva', 'Insulin', 'Cataracts', 'Methadone', 'Diabetes', 'Blanching', 'Thirsty', 'Photosensitization', 'Polyuria', 'Psoriasis', 'Virilism', 'Comatose', 'Sleeplessness', 'Neuropathy', 'Tuberculosis', 'Obesity', 'Thromboembolism', 'Pneumoniae', 'Cardura', 'Cirrhosis', 'Anemia', 'Osteoporosis', 'Garlic', 'Dyslipidemia', 'Fibrillation', 'Cefoxitin', 'Deafness', 'Penicillin', 'Hemochromatosis', 'Ischemias', 'Pallor', 'Rabies', 'Chyluria', 'Alcohol', 'Codeine', 'Coumadin', 'Vancocin', 'Coccidioses', 'Aneurysm', 'Flatulence', 'Cefazolin', 'Encephalitis', 'Hemophilia', 'Arthritis', 'Urethrorrhea', 'Anovulation', 'Albumin', 'Snoring', 'Haemorrhoids', 'Epilepsy', 'Arteriosclerosis', 'Myelosuppression', 'Hernias

In [98]:
data2['Term2'].value_counts()

Insulin        9
Coumadin       8
Ethanol        7
Tylenol        7
Headache       6
              ..
Mannerism      1
Cardura        1
Rogaine        1
Carsickness    1
Protonix       1
Name: Term2, Length: 281, dtype: int64

In [99]:
data2_words2= data2['Term2']

In [100]:
data2_token2 = [word_tokenize(word) for word in data2_words2]

print(data2_token2)

[['Fibrillation'], ['Dilantin'], ['Tylenol'], ['Hyperthyroidism'], ['Nausea'], ['Histoplasmosis'], ['Plavix'], ['Ipecac', 'Syrup'], ['Insulin'], ['Dyspnea'], ['Incontinence'], ['Ethanol'], ['Hyperacusis'], ['Cholestyramine'], ['Famvir'], ['Vomiting'], ['Chyluria'], ['Ethanol'], ['Malaria'], ['Diabetes'], ['Cromolyn'], ['Hunger'], ['Serevent'], ['Folic', 'Acid'], ['Glucagen'], ['Wheezing'], ['Ultram'], ['Psoriasis'], ['Pallor'], ['Lasix'], ['Aphonia'], ['Chyluria'], ['Meningism'], ['Drooling'], ['Hepatitis'], ['Agitation'], ['Glucophage'], ['Arthritis'], ['Clubbing'], ['Diabetes'], ['Hemoptysis'], ['Ceftiaxone'], ['Hytrin'], ['Glucophage'], ['Banana', 'Bag'], ['Carbatrol'], ['Insulin'], ['Reticulocytosis'], ['Carbatrol'], ['Prednisolone'], ['Dilantin'], ['Fish', 'Oil'], ['Fish', 'Oil'], ['Angina'], ['Ceftiaxone'], ['Ataxia'], ['Cefazolin'], ['Insulin'], ['Anosmia'], ['Mittelschmerz'], ['Aspirin'], ['Crowning'], ['Cholestasis'], ['Cholestyramine'], ['Acne'], ['Adenitis'], ['Menouria'], [

In [101]:
mymodel_3= Word2Vec(data2_token2, min_count=1)
print(mymodel_3)

Word2Vec(vocab=290, size=100, alpha=0.025)


In [102]:
words_3 = list(mymodel_3.wv.vocab)
print(words_3)

['Fibrillation', 'Dilantin', 'Tylenol', 'Hyperthyroidism', 'Nausea', 'Histoplasmosis', 'Plavix', 'Ipecac', 'Syrup', 'Insulin', 'Dyspnea', 'Incontinence', 'Ethanol', 'Hyperacusis', 'Cholestyramine', 'Famvir', 'Vomiting', 'Chyluria', 'Malaria', 'Diabetes', 'Cromolyn', 'Hunger', 'Serevent', 'Folic', 'Acid', 'Glucagen', 'Wheezing', 'Ultram', 'Psoriasis', 'Pallor', 'Lasix', 'Aphonia', 'Meningism', 'Drooling', 'Hepatitis', 'Agitation', 'Glucophage', 'Arthritis', 'Clubbing', 'Hemoptysis', 'Ceftiaxone', 'Hytrin', 'Banana', 'Bag', 'Carbatrol', 'Reticulocytosis', 'Prednisolone', 'Fish', 'Oil', 'Angina', 'Ataxia', 'Cefazolin', 'Anosmia', 'Mittelschmerz', 'Aspirin', 'Crowning', 'Cholestasis', 'Acne', 'Adenitis', 'Menouria', 'Antabuse', 'Narcan', 'Glucotrol', 'Meningitis', 'Coumadin', 'Osteoporosis', 'Toprol-XR', 'Keflex', 'Thrombus', 'Thromboembolism', 'Arthralgia', 'Tremor', 'Carboplatin', 'Heparin', 'Stridor', 'Coreg', 'Infertility', 'Phenobarbital', 'Alcohol', 'Spasm', 'Loperamide', 'Snoring', 

In [78]:
#Trying to find out most similar words for the words like "glaucoma","fibrillation","carbatrol","dilantin","hyperthyroidism","cardiomyopathy" and "septicemia" 

In [ ]:
#The vector of the above words will be compared with the vector of words from the ClinNotes, and whichever comes closer will give an output.

In [103]:
mymodel.most_similar('glaucoma')

[('attack', 0.9862457513809204),
 ('negative', 0.983184814453125),
 ('gain', 0.9829369783401489),
 ('above.', 0.9829050302505493),
 ('cholesterol', 0.9813812971115112),
 ('glaucoma.', 0.980632483959198),
 ('symptoms', 0.980423092842102),
 ('cyanotic', 0.9802300930023193),
 ('chills.', 0.9802271723747253),
 ('failure', 0.9802235960960388)]

In [104]:
mymodel.most_similar('fibrillation')

[('failure', 0.9965488910675049),
 ('congestive', 0.9942981600761414),
 ('congenital', 0.986838161945343),
 ('rheumatic', 0.9849531650543213),
 ('-pound', 0.9837476015090942),
 ('attack', 0.9836606979370117),
 ('increased.', 0.9825565218925476),
 ('white', 0.9819899201393127),
 ('arthritis.', 0.9813177585601807),
 ('failure.,3', 0.9789136648178101)]

In [105]:
mymodel.most_similar('carbatrol')

[('tablet', 0.969409704208374),
 ('phenergan', 0.9686115980148315),
 ('b.i.d', 0.9674776196479797),
 ('tablets', 0.96677565574646),
 ('atropine', 0.9663378000259399),
 ('daily.,3', 0.9658218622207642),
 ('la', 0.9651377201080322),
 ('qh', 0.9650895595550537),
 ('q.d', 0.9650436639785767),
 ('daily', 0.9647830128669739)]

In [106]:
mymodel.most_similar('dilantin')

[('doses', 0.9984338283538818),
 ('discontinued', 0.9983612895011902),
 ('started', 0.9981629848480225),
 ('nitroglycerin', 0.9981359243392944),
 ('friday', 0.9979586601257324),
 ('keppra', 0.997909665107727),
 ('acid', 0.9977120161056519),
 ('later', 0.9977063536643982),
 ('prescription', 0.9976900815963745),
 ('tylenol', 0.997543454170227)]

In [107]:
mymodel.most_similar('hyperthyroidism')

[('language', 0.9865998029708862),
 ('future', 0.9864064455032349),
 ('always', 0.9863561391830444),
 ('correlation', 0.9863020181655884),
 ('ha', 0.986296534538269),
 ('trouble', 0.9862573146820068),
 ('tried', 0.9862568378448486),
 ('hx', 0.9862452149391174),
 ('university', 0.9861553907394409),
 ('nose', 0.9861510396003723)]

In [108]:
mymodel.most_similar('cardiomyopathy')

[('dementia', 0.9988489151000977),
 ('diagnosed', 0.9983612298965454),
 ('questionable', 0.998272180557251),
 ('dysphagia', 0.9980999231338501),
 ('bleed', 0.9979890584945679),
 ('disorder', 0.997829258441925),
 ('maternal', 0.9978153705596924),
 ('vertigo', 0.9977684617042542),
 ('progression', 0.9976323246955872),
 ('early', 0.9976207613945007)]

In [109]:
mymodel.most_similar('septicemia')

[('transaminases', 0.9932562112808228),
 ('tilt', 0.9929028153419495),
 ('lose', 0.9926412105560303),
 ('mucus', 0.9924490451812744),
 ('hydrocephalus', 0.9923726916313171),
 ('skills', 0.9923716187477112),
 ('visual', 0.9922685623168945),
 ('tests', 0.9921871423721313),
 ('toilet', 0.9920949339866638),
 ('looks', 0.992079496383667)]

In [110]:
#Separate X and Y and perform train test split, 70-30

In [111]:
len(data_notes_clean)

818

In [113]:
X = word_list
y = NLP_clin_note.category

In [103]:
#train-test split

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [115]:
#term matrix using TfIdf

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [117]:
DTM = TfidfVectorizer(max_features = 4000)

In [118]:
len(X_train), len(X_test)

(572, 246)

In [121]:
#term matrix using bag of words

In [122]:
X_train_bagofwords = DTM.fit_transform(X_train)

In [123]:
X_test_bagofwords = DTM.transform(X_test)

In [124]:
X_train_bagofwords.shape, X_test_bagofwords.shape

((572, 4000), (246, 4000))

In [125]:
#Model Building

In [126]:
from sklearn import svm

In [127]:
classifier_linear = svm.SVC(kernel='linear')

In [129]:
classifier_linear.fit(X_train_bagofwords, y_train)

SVC(kernel='linear')

In [130]:
y_train_prediction = classifier_linear.predict(X_train_bagofwords)

In [131]:
y_train_prediction[:8]

array(['Neurology', 'Cardiovascular / Pulmonary', 'Gastroenterology',
       'Neurology', 'Cardiovascular / Pulmonary', 'Gastroenterology',
       'Gastroenterology', 'Cardiovascular / Pulmonary'], dtype=object)

In [132]:
from sklearn.metrics import classification_report

In [133]:
print(classification_report(y_train, y_train_prediction))

                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       0.99      0.99      0.99       264
          Gastroenterology       0.99      1.00      1.00       155
                 Neurology       0.99      0.98      0.99       153

                  accuracy                           0.99       572
                 macro avg       0.99      0.99      0.99       572
              weighted avg       0.99      0.99      0.99       572



In [134]:
#Adjusting the class weights to improve the recall for the label

In [135]:
classifier_linear = svm.SVC(kernel='linear', class_weight="balanced")

In [136]:
classifier_linear.fit(X_train_bagofwords, y_train)

SVC(class_weight='balanced', kernel='linear')

In [137]:
y_train_prediction = classifier_linear.predict(X_train_bagofwords)

In [138]:
print(classification_report(y_train, y_train_prediction))

                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       1.00      0.99      0.99       264
          Gastroenterology       0.99      1.00      0.99       155
                 Neurology       0.99      0.99      0.99       153

                  accuracy                           0.99       572
                 macro avg       0.99      0.99      0.99       572
              weighted avg       0.99      0.99      0.99       572



In [139]:
#improved result

In [140]:
#Apply the Hyper parameter tuning
#"class_weights" a hyperparameters for tuneing the SVM.
#Identify hyper-parameters for the SVM classifier

In [141]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [142]:
# Grid search and random search 
param_grid = {
    'C': [0.1, 1, 10,1000, 10000, 100000]
}

In [143]:
classifier_svm = svm.SVC(random_state=40, class_weight="balanced", kernel="linear")

In [144]:
# Invoke the grid search model
Grid_Search = GridSearchCV(estimator = classifier_svm, param_grid = param_grid, 
                          cv = StratifiedKFold(5), n_jobs = -1, verbose = 1, scoring = "recall" )

In [145]:
Grid_Search.fit(X_train_bagofwords, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVC(class_weight='balanced', kernel='linear',
                           random_state=40),
             n_jobs=-1, param_grid={'C': [0.1, 1, 10, 1000, 10000, 100000]},
             scoring='recall', verbose=1)

In [146]:
Grid_Search.best_estimator_

SVC(C=0.1, class_weight='balanced', kernel='linear', random_state=40)

In [147]:
y_train_prediction = Grid_Search.best_estimator_.predict(X_train_bagofwords)

In [148]:
print(classification_report(y_train, y_train_prediction))

                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       0.97      0.97      0.97       264
          Gastroenterology       0.97      0.97      0.97       155
                 Neurology       0.96      0.95      0.96       153

                  accuracy                           0.97       572
                 macro avg       0.97      0.97      0.97       572
              weighted avg       0.97      0.97      0.97       572



In [149]:
#Most prominent terms in the category

In [150]:
y_train_prediction = Grid_Search.best_estimator_.predict(X_test_bagofwords)

In [151]:
Category_notes_Neurology = pd.Series(X_test)[y_train_prediction=='Neurology'].values

In [152]:
word_list_1= []
for word in Category_notes_Neurology:
    word_list_1.extend(word_tokenize(word))

In [153]:
cts_Neurology = Counter(word_list_1)

In [154]:
cts_Neurology.most_common(10)

[('.', 925),
 ('patient', 214),
 ('right', 183),
 ('normal', 175),
 ('left', 174),
 ('history', 167),
 ('unremarkable', 100),
 ('pain', 100),
 ('exam', 90),
 ('also', 80)]

In [155]:
Category_notes_Cardiovascular = pd.Series(X_test)[y_train_prediction=='Cardiovascular / Pulmonary'].values

In [156]:
word_list_2= []
for word in Category_notes_Cardiovascular:
    word_list_2.extend(word_tokenize(word))

In [157]:
cts_Cardiovascular= Counter(word_list_2)

In [158]:
cts_Cardiovascular.most_common(10)

[('.', 894),
 ('patient', 469),
 ('left', 337),
 ('right', 275),
 ('artery', 230),
 ('history', 205),
 ('mg', 190),
 ('coronary', 183),
 ('procedure', 179),
 ('chest', 161)]

In [159]:
Category_notes_Gastroenterology=pd.Series(X_test)[y_train_prediction=='Gastroenterology'].values

In [160]:
word_list_3= []
for word in Category_notes_Gastroenterology :
    word_list_3.extend(word_tokenize(word))

In [161]:
cts_Gastroenterology= Counter(word_list_3)

In [162]:
cts_Gastroenterology.most_common(10)

[('.', 415),
 ('patient', 248),
 ('procedure', 167),
 ('placed', 112),
 ('history', 95),
 ('well', 75),
 ('colon', 72),
 ('abdomen', 70),
 ('normal', 70),
 ('stomach', 66)]